In [1]:
# there are too many itineraries to have a tool for each itinerary. I need to have a single
# Q&A that will check the which itinerary to look up
# pip install "langchain>=0.0.27"

In [2]:
import pinecone
from google.cloud import bigquery
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Pinecone
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import BigQueryLoader
from pydantic import BaseModel
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from typing import Type
from pydantic import BaseModel, Field
from langchain.tools import BaseTool

from dotenv import load_dotenv

import os

# Load the environment variables from the .env file
load_dotenv()

# Access environment variables
api_key = os.getenv("OPENAI_KEY")
pinecone_key = os.getenv('PINECONE_API_KEY')
pinecone_env = os.getenv('PINECONE_ENV')

/home/stuart/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
embeddings = OpenAIEmbeddings(openai_api_key=api_key)

In [4]:

pinecone.init(api_key=pinecone_key,
              environment=pinecone_env)
# pinecone.create_index("chatbot", dimension=1536)

In [5]:
PROJECT = "wagon-bootcamp-377120"
DATASET = "g_adventures_dataset"
TABLE = "one_month"

query = f"""
    SELECT 
      DISTINCT tour_name
    FROM {PROJECT}.{DATASET}.{TABLE}
    """

client = bigquery.Client(project="wagon-bootcamp-377120")
query_job = client.query(query)
result = query_job.result()
df = result.to_dataframe()

In [6]:
tour_name_list = df['tour_name']

In [7]:
from langchain.agents.agent_toolkits import create_retriever_tool
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS


embeddings = OpenAIEmbeddings(openai_api_key=api_key)
vector_db = FAISS.from_texts(tour_name_list, embeddings)
retriever = vector_db.as_retriever(k=1)

retriever_tool = create_retriever_tool(
        retriever,
        name='itinerary_name_search',
        description='use to learn the correct spelling of an itinerary name'
    )



In [8]:
def question_function(att: str, question: str):
    
    embeddings = OpenAIEmbeddings(openai_api_key=api_key)
    
    pinecone_index = pinecone.Index("chatbot")
    vectorstore = Pinecone(pinecone_index, embeddings.embed_query, "text")
    
    filters = {"tour_name": att}

    qa = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=vectorstore.as_retriever(search_kwargs={'filter': filters})
    )

    response = qa.run(question)
    return response


class Itineraryname(BaseModel):
    """Inputs for my_profile function"""
    itinerary_name: str = Field(description="the correct spelling of an itinerary name")
    question: str = Field(description="the question to be passed to the model")

class ItineraryQuestions(BaseTool):
    name = "itinerary_question"
    description = """
        Needed when you want to ask a question about an itinerary.
        Use this tool once you have used the retriever_tool to get the correct name
        """
    args_schema: Type[BaseModel] = Itineraryname

    def _run(self, itinerary_name: str, question: str):
        profile_detail_attribute = question_function(itinerary_name, question)
        return profile_detail_attribute

    def _arun(self, itinerary_name: str):
        raise NotImplementedError(
            "itinerary_question does not support async calls")

In [10]:
custom_tool_list = [retriever_tool, ItineraryQuestions()]

In [11]:
llm = ChatOpenAI(
    openai_api_key=api_key,
    model_name='gpt-3.5-turbo',
    temperature=0.0
)

In [12]:
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

SALES_AGENT_TOOLS_PROMPT = """You are being asked a question about an itinerary.
The first thing you need to do is to get the correct itinerary name using the itinerary_name_search tool.
Once you hvae the correct itinerary name use the itinerary_question tool to answer the question.

"""

final_prompt = ChatPromptTemplate.from_messages([
  ("system", SALES_AGENT_TOOLS_PROMPT),
  ("user", "{input}"),
  MessagesPlaceholder(variable_name="agent_scratchpad"),
  #MessagesPlaceholder(variable_name="chat_history")  
])

In [16]:
from langchain.agents import OpenAIFunctionsAgent, AgentExecutor

# Create the agent
agent = OpenAIFunctionsAgent(llm=llm,
                             tools=custom_tool_list,
                             prompt=final_prompt
                            )


# Run the agent with the actions
agent_executor = AgentExecutor(agent=agent,
                               tools=custom_tool_list,
                               verbose=True,
                               max_iterations=5)


In [17]:
# %%time
# response = agent_executor.run("how long is the Kenya & Uganda Gorilla Overland itinerary?")
# response

In [18]:
%%time
response = agent_executor.run("which itinerary is longer the highlights of madagascar or the wonders of namibia?")
response



> Entering new AgentExecutor chain...

Invoking: `itinerary_name_search` with `{'query': 'highlights of madagascar'}`


[Document(page_content='Highlights of Madagascar'), Document(page_content='Southern Africa Highlights'), Document(page_content='Highlights of Morocco'), Document(page_content='Highlights of New Zealand')]
Invoking: `itinerary_name_search` with `{'query': 'wonders of namibia'}`


[Document(page_content='Wonders of Namibia'), Document(page_content='Discover Namibia & Victoria Falls'), Document(page_content='Discover Kruger & Namibia'), Document(page_content='Wonders of Bhutan')]
Invoking: `itinerary_question` with `{'itinerary_name': 'Highlights of Madagascar', 'question': 'How long is the Highlights of Madagascar itinerary?'}`




/home/stuart/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/langchain/vectorstores/pinecone.py:59: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


The Highlights of Madagascar itinerary is 14 days long.
Invoking: `itinerary_question` with `{'itinerary_name': 'Wonders of Namibia', 'question': 'How long is the Wonders of Namibia itinerary?'}`




/home/stuart/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/langchain/vectorstores/pinecone.py:59: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


The Wonders of Namibia itinerary is 10 days long.The Highlights of Madagascar itinerary is longer than the Wonders of Namibia itinerary. The Highlights of Madagascar itinerary is 14 days long, while the Wonders of Namibia itinerary is 10 days long.

> Finished chain.
CPU times: user 241 ms, sys: 0 ns, total: 241 ms
Wall time: 18.6 s


'The Highlights of Madagascar itinerary is longer than the Wonders of Namibia itinerary. The Highlights of Madagascar itinerary is 14 days long, while the Wonders of Namibia itinerary is 10 days long.'